# Agent routing via available tooling

When configuring a `Team`, you normally provide a brie description of each `Agent` to the orchestrator will have enough information to pick the best-suited one next in the conversation.

Usually, a good description should include the agent purpose and its main capabilities. However, this may get more complex and less manageable as the agents evolve and their purpose is adjusted, leading to the change of improper routing due to the orchestrator not having accurate information.

A complementary approach is to also tell the orchestrator to look at each `Agent` available tools. This can also help to route to an agent given a specific user asks that maps to a function call.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Add the parent directory to sys.path
import sys, os
sys.path.append(os.path.abspath(os.path.join('../vanilla_aiagents')))

from vanilla_aiagents.agent import Agent
from vanilla_aiagents.workflow import Workflow
from vanilla_aiagents.team import Team
from vanilla_aiagents.llm import AzureOpenAILLM
from dotenv import load_dotenv

load_dotenv(override=True)

In [ ]:
llm = AzureOpenAILLM({
    "azure_deployment": os.getenv("AZURE_OPENAI_MODEL"),
    "azure_endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.getenv("AZURE_OPENAI_KEY"),
    "api_version": os.getenv("AZURE_OPENAI_API_VERSION"),
})

# Set logging to debug for Agent, User and Workflow
import logging

# Set logging to debug for Agent, User, and Workflow
logging.basicConfig(level=logging.INFO)
logging.getLogger("vanilla_aiagents.agent").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.workflow").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.llm").setLevel(logging.DEBUG)

In [ ]:
# Agent description here is purposely vague
first = Agent(id="first", llm=llm, description="Agent1", system_message = """You are part of an AI process
Your task is to support the user inquiry by providing the user profile.
""")

@first.register_tool(name="get_product_info", description="Get the product info")
def get_product_info():
    return "Product info: Product A"

# And here as well
second = Agent(id="second", llm=llm, description="Agent2", system_message = """You are part of an AI process
Your task is to support the user inquiry by providing the user balance.
""")

@second.register_tool(name="get_user_balance", description="Get the user balance")
def get_user_balance():
    return "User balance: $100"

# include_tools_descriptions is set to True
flow = Team(id="flow", description="", members=[first, second], llm=llm, stop_callback=lambda conv: len(conv.messages) > 2, include_tools_descriptions=True, use_structured_output=False)


In [ ]:
workflow = Workflow(askable=flow)
workflow.restart()

workflow.run("Which is my current balance?")

In [ ]:
workflow.conversation.messages